## Profiling report and KNN Imputation for status and value features

#### Libraries

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.impute import KNNImputer

#### DF Reading

In [2]:
# Read CSVs
df_training = pd.read_csv('../data/training.csv')
df_test     = pd.read_csv('../data/testing.csv')
DF          = pd.concat([df_training, df_test], sort=False)

In [3]:
df_training.head(3)

,city,website,employees,status,value
0,New York,http://www.ejmequities.com/,101-250,private,0
1,Las Vegas,http://www.digipath.com,11-50,public,0
2,Mountain View,http://www.videogram.com,1-10,private,0


## Pandas Profiling
- The final result can be seen in 'Profiling_report.html'

In [ ]:
profile = ProfileReport(df_training, title="Profiling Report", explorative=True)
#profile.to_file("Profiling_report.html")

## KNN Imputation

### Categorical Variables

In [5]:
# Parse Object variables as Categorical for further imputation
DF['employees'] = pd.Categorical(DF.employees)
DF['city']      = pd.Categorical(DF.city)
DF['status']    = pd.Categorical(DF.status)
DF.dtypes

city         category
website        object
employees    category
status       category
value         float64
dtype: object

In [6]:
# Website variable is removed for cleaner analysis
DF_mini = DF[['city', 'employees', 'status', 'value' ]]
DF_mini.head()

,city,employees,status,value
0,New York,101-250,private,0.0
1,Las Vegas,11-50,public,0.0
2,Mountain View,1-10,private,0.0
3,Glen Allen,11-50,private,0.0
4,Houston,11-50,private,0.0


In [7]:
# Dummies encoding for City and Employees variables
categorical_cols = ['city', 'employees'] 
DF_dummy         = pd.get_dummies(DF_mini, columns = categorical_cols)

### Imputation for status and value columns

In [8]:
# Label Encoding for Status variable
from sklearn import preprocessing
label_encoder      = preprocessing.LabelEncoder()
DF_dummy['status'] = label_encoder.fit_transform(DF_dummy['status'])
# Replace label '3' for NaN
DF_dummy['status'] = DF_dummy['status'].replace(3, np.NaN)

### KNN Imputer

In [10]:
imputer    = KNNImputer(n_neighbors=1)
DF_imputed = imputer.fit_transform(DF_dummy)
df         = pd.DataFrame(imputer.fit_transform(DF_imputed), columns = DF_dummy.columns)

In [12]:
# Take the last 2000 values to match original testing.csv
df_test_final = df.iloc[2000:].reset_index(drop=True)

In [13]:
imputated_cols = df_test_final[['value', 'status']]
df_test_final  = pd.concat([df_test, imputated_cols], axis=1, join='inner')

In [14]:
# Finally, parse status column again as string
df_test_final['status'] = df_test_final['status'].astype(int).astype(str)
df_test_final['status'] = np.where(df_test_final['status']=='1', 'private', df_test_final['status'])
df_test_final['status'] = np.where(df_test_final['status']=='2', 'public', df_test_final['status'])
df_test_final['status'] = np.where(df_test_final['status']=='0', 'delisted', df_test_final['status'])
df_test_final['value'] = df_test_final.value.astype(int)


In [15]:
# Last value counts: status
df_test_final.status.value_counts()

private     1738
public       239
delisted      23
Name: status, dtype: int64

In [16]:
# Last value counts: value
df_test_final.value.value_counts()

0    1675
1     325
Name: value, dtype: int64

In [17]:
# Final df format
df_test_final

,city,website,employees,value,status
0,Kansas City,http://www.lendingstandard.com,11-50,0,private
1,Newport Beach,https://www.cudasign.com/,11-50,0,private
2,Oakland,https://fivetran.com,251-500,0,private
3,New York,http://www.powerspike.tv,11-50,1,private
4,Sacramento,https://www.rhombussystems.com,11-50,0,private
...,...,...,...,...,...
1995,Philadelphia,https://www.vybe.care/,11-50,0,private
1996,Boston,http://www.qmenta.com,11-50,0,private
1997,San Francisco,https://www.bitski.com/,1-10,1,private
1998,Alpharetta,https://cmon.com/,11-50,0,private


### Export as CSV file

In [ ]:
#df_test_final.to_csv('../processed/testing_filled.csv', index=False)